In [ ]:
#
# Initialization
#
import os
import sys

import ipynbname
from pathlib import Path

# Set notebook's src module path. Note that you may have to update your IDE's project settings to do the same for the
#  local library imports to work the same

MODULE_PATH = ipynbname.path().parent.parent
sys.path.append(str(MODULE_PATH))

# Keep paths consistent throughout notebook
os.chdir(MODULE_PATH)

# This should always be `./src`
print(f"Current working directory [{os.getcwd()}]")

# Place all local artifacts in a disposable, git-ignored directory
local_artifact_dir = Path(os.getcwd()).parent / "out"
local_artifact_dir.mkdir(parents=True, exist_ok=True)

# Autoreload imports at the beginning of cell execution.
#  https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
#
# Setup utils
#

from utils.logger_util import LoggerUtil
from utils.utils import Utils
from utils import config

LOGGER = LoggerUtil(config.MODEL_ID, local_artifact_dir / "logs")
UTILS = Utils(LOGGER)

UTILS.describe_env()

LOCAL_INSTANCE = 'local'

In [ ]:
from data_processing.aws_handler import AWSHandler
from sagemaker import LocalSession

AWS_HANDLER = AWSHandler(LOGGER, config.MODEL_ID)

sagemaker_session = LocalSession()
sagemaker_session.config = {
    'local': {
        'local_code': True
    }
}

In [ ]:
#
# Upload serialized model
#

from sagemaker.mxnet import MXNetModel

# Place the model (`model.tar.gz` contents) that you would like to run locally inside this directory.
# This will recompress them in the appropriate format
model_dir_path = local_artifact_dir / "local_deploy_model" / "model"
model_dir_path.mkdir(parents=True, exist_ok=True)
LOGGER.log(f"Model dir is [{model_dir_path}]")

# This is stupid, but we have to upload the model to S3 for Sagemaker, even though Sagemaker will download it locally
# anyways to run it. Their Model class does not support local paths
s3_model_path = AWS_HANDLER.upload_model_to_s3(model_dir_path)

In [ ]:
#
# Deploy model
#

import logging

LOGGER.log(f"Removing preexisting models and endpoints")
try:
    sagemaker_session.sagemaker_client.delete_model(config.MODEL_NAME)
    sagemaker_session.sagemaker_client.delete_endpoint(config.MODEL_NAME)
    sagemaker_session.sagemaker_client.delete_endpoint_config(config.MODEL_NAME)
except:
    pass

LOGGER.log(f"Creating model")
model = MXNetModel(
    model_data=s3_model_path,
    name=config.MODEL_NAME,
    entry_point='deepar_dynamic_real.py',
    source_dir=os.getcwd(),
    role=config.SM_ROLE,
    framework_version='1.7.0',  # Should be the same mxnet X.X.X version found in requirements.txt
    py_version='py3',
    sagemaker_session=sagemaker_session,
    container_log_level=logging.DEBUG
)

LOGGER.log(f"Deploying model")
predictor = model.deploy(
    instance_type = LOCAL_INSTANCE,
    endpoint_name = config.MODEL_NAME,
    initial_instance_count = 1)
LOGGER.log(sagemaker_session.sagemaker_client.describe_model(config.MODEL_NAME))
LOGGER.log(sagemaker_session.sagemaker_client.describe_endpoint_config(config.MODEL_NAME))
LOGGER.log(f"Predictor metadata [{predictor.__dict__}]")
LOGGER.log(f"http://localhost:{sagemaker_session.sagemaker_runtime_client.serving_port}/invocations")
